In [5]:
import os
import re
import nltk
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fahad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fahad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\fahad\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
def extract_data(folder, new_folder):
    files = os.listdir(folder)
    for file in files:
        path = os.path.join(os.getcwd(), folder, file)
        with open(path) as fp:
            soup = BeautifulSoup(fp, 'html.parser')
            text = soup.findAll("text")[0].text
            title = soup.findAll("title")[0].text
            final_text = title + " " + text
        new_file_path = os.path.join(os.getcwd(), new_folder, file)
        with open(new_file_path, "w") as fw:
            fw.write(final_text)
            fw.close()

In [7]:
new_folder = 'Dataset'
try:
    os.mkdir(new_folder)
except:
    print(new_folder, " already exists.")
extract_data('CSE508_Winter2023_Dataset', new_folder)


Dataset  already exists.


In [8]:
def preprocess(text):

    text = text.lower()

    tokens = word_tokenize(text)
    
    final = [word for word in tokens if word not in stop_words]

    tokens = [word for word in final if word not in string.punctuation]
     
    final = [word for word in tokens if len(re.findall(r'\s+', word)) == 0]
    
    return final

In [9]:
L = []
idtoName = {}
files = os.listdir("Dataset")
for i, file in enumerate(files):
    idtoName[i] = file
    path = os.path.join(os.getcwd(), "Dataset", file)
    with open(path) as fp:
        text = fp.read()
        L.append(preprocess(text))

In [10]:
def unigram_inverted_index(doc_list):
    uni_inv_idx = {}
    for doc_id, tokens in enumerate(doc_list):
        for idx, token in enumerate(tokens):
            if token in uni_inv_idx:
                if doc_id not in uni_inv_idx[token]:
                    uni_inv_idx[token].append(doc_id)
            else:
                uni_inv_idx[token] = [doc_id]

    return uni_inv_idx

In [11]:
uni_inv_idx = unigram_inverted_index(L)

<h3>Saving to Pickle

In [15]:
import pickle
filehandler = open("uni_inv_idx.obj","wb")
pickle.dump(uni_inv_idx, filehandler)
filehandler.close()

<h3>Loading from Pickle

In [16]:
file = open("uni_inv_idx.obj",'rb')
uni_inv_idx = pickle.load(file)
file.close()

In [17]:
def andQuery(L1, L2):
    ans = []
    comparison = 0
    len1, len2 = len(L1), len(L2)
    
    i , j = 0, 0
    while i < len1 and j < len2:
        if L1[i] == L2[j]:
            ans.append(L1[i])
            i += 1
            j += 1
        elif L1[i] < L2[j]:
            i += 1
        else:
            j += 1
        comparison += 1
    
    return ans, comparison

In [18]:
def andNotQuery(L1, L2):
    _L2 = [i for i in range(1400) if i not in L2]   
    return andQuery(L1, _L2)

In [19]:
def orQuery(L1, L2):
    ans = []
    comparison = 0
    len1, len2 = len(L1), len(L2)
    
    i , j = 0, 0
    while i < len1 and j < len2:
        if L1[i] == L2[j]:
            ans.append(L1[i])
            i += 1
            j += 1
        elif L1[i] < L2[j]:
            ans.append(L1[i])
            i += 1
        else:
            ans.append(L2[j])
            j += 1
            
        comparison += 1
    
    while i < len1:
        ans.append(L1[i])
        i += 1
    
    while j < len2:
        ans.append(L2[j])
        j += 1

    return ans, comparison

In [20]:
def orNotQuery(L1, L2):
    _L2 = [i for i in range(1400) if i not in L2]   
    return orQuery(L1, _L2)

In [21]:
def process_query(i, query, operator, comparisons):
    if i == len(operator):
        return query, comparisons
    
    res = []
    comp = 0

    if 'OR' in operator[i] and 'NOT' not in operator[i]:
        # print('OR')
        res, comp = orQuery(query[0], query[1])

    if 'AND' in operator[i] and 'NOT' not in operator[i]:
        # print('AND')
        res, comp = andQuery(query[0], query[1])

    if 'AND NOT' in operator[i]:
        # print('AND NOT')
        res, comp = andNotQuery(query[0], query[1])
    
    if 'OR NOT' in operator[i]:
        # print('OR NOT')
        res, comp = orNotQuery(query[0], query[1])
    
    del query[ : 2]
    query.insert(0, res)
    comparisons += comp
    
    return process_query(i + 1, query, operator, comparisons)


In [22]:
N = int(input())
count = 1
while count <= N:
    ip1 = input()
    op = input()
    op = op.split(',')
    ip1 = preprocess(ip1)
    query = [uni_inv_idx[i] if i in uni_inv_idx else [] for i in ip1]

    if len(query) != len(op) + 1:
        print("Inappropriate query !!!")
        break
    
    sent = ""
    p, q = 0, 0
    for idx in range(len(query) + len(op)):
        if idx % 2 == 0:
            sent += ip1[p] + " "
            p += 1
        else:
            sent += op[q] + " "
            q += 1
    
    print(f"Query {count}: ", sent)
    comparisons = 0
    query, comparisons = process_query(0, query, op, comparisons)
    docs = [idtoName[i] for i in query[0]]
    print(f"Number of documents retrieved for query {count}: ", len(query[0]))
    print(f"Names of the documents retrieved for query {count}: ", docs)
    print(f"Number of comparisons required for query {count}: ", comparisons)
    count += 1

Query 1:  one OR two AND three 
Number of documents retrieved for query 1:  36
Names of the documents retrieved for query 1:  ['cranfield0006', 'cranfield0059', 'cranfield0064', 'cranfield0140', 'cranfield0187', 'cranfield0266', 'cranfield0364', 'cranfield0454', 'cranfield0456', 'cranfield0476', 'cranfield0536', 'cranfield0582', 'cranfield0609', 'cranfield0620', 'cranfield0641', 'cranfield0660', 'cranfield0672', 'cranfield0673', 'cranfield0679', 'cranfield0750', 'cranfield0773', 'cranfield0785', 'cranfield0791', 'cranfield0844', 'cranfield0856', 'cranfield0948', 'cranfield1036', 'cranfield1049', 'cranfield1092', 'cranfield1125', 'cranfield1144', 'cranfield1145', 'cranfield1325', 'cranfield1351', 'cranfield1357', 'cranfield1392']
Number of comparisons required for query 1:  888
